In [146]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import jieba
from jieba import lcut, analyse, Tokenizer
from wordcloud import WordCloud, STOPWORDS
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors

In [147]:
def Openfile(path):
    f = open(path, "r", encoding="utf-8")
    result = f.read()
    f.close()
    return result
def Openfile_sp(path):
    f = open(path, "r", encoding="utf-8")
    result = f.readlines()
    f.close()
    return result

In [148]:
# open file
stopwords_file = 'datafile/stop_words.txt'
jieba.set_dictionary('./dict.txt.big.txt')
jieba.load_userdict('datafile/user_dict.txt')
jieba.analyse.set_stop_words('datafile/stop_words.txt')
tigerair_file = 'datafile/tigerair_rawtext.txt'
tigerair_text = Openfile_sp(tigerair_file)

Building prefix dict from D:\WORK5\TechProject\TechProject2019\dict.txt.big.txt ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.u15c153cb84f92c395787873e58f7016c.cache
Loading model cost 0.989 seconds.
Prefix dict has been built succesfully.


In [153]:
dec = Openfile_sp(stopwords_file)
stopwords = set()
for i in range(len(dec)):
    stopwords.add(dec[i].strip())


In [154]:
result = []
for i in range(len(tigerair_text)):
    r = lcut(tigerair_text[i], cut_all=False, HMM=False)
    result.append(r)

In [155]:
adj_result = []
for i in range(len(result)):
    par = []
    for j in range(len(result[i])):
        if len(result[i][j]) > 1:
            if result[i][j] not in stopwords:
                par.append(result[i][j])
    adj_result.append(par)

In [156]:
T_model = word2vec.Word2Vec(adj_result, size=300, min_count=10)
T_model.save("model/Tigerair_model.bin")

In [187]:
model = word2vec.Word2Vec.load("model/Tigerair_model.bin")
word_vectors = model.wv

In [188]:
res = word_vectors.wv.most_similar('虎迷', topn = 20)

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [189]:
res

[('分享', 0.902275562286377),
 ('可以', 0.898814857006073),
 ('準備', 0.8933571577072144),
 ('就是', 0.8924463987350464),
 ('體驗', 0.8862548470497131),
 ('住宿', 0.8798786401748657),
 ('機會', 0.8730891942977905),
 ('朋友', 0.8655794262886047),
 ('旅行', 0.8628038763999939),
 ('免費', 0.8601059913635254),
 ('自己', 0.856334924697876),
 ('機票', 0.8543732762336731),
 ('出國', 0.8455445170402527),
 ('前往', 0.8442637324333191),
 ('不要', 0.8434194326400757),
 ('折價券', 0.8433970212936401),
 ('參加', 0.8433293104171753),
 ('一張', 0.8369359970092773),
 ('現場', 0.8341183662414551),
 ('不用', 0.8272459506988525)]

In [182]:
# test part
key_word = 'datafile/tigerair_keywords.csv'
df = pd.read_csv(key_word)
key = set()
df1 = df['key'].tolist()
for i in range(len(df1)):
    key.add(df1[i])
    
key_result = []
for i in range(len(adj_result)):
    par = []
    for j in range(len(adj_result[i])):
        if adj_result[i][j] in key:
            par.append(adj_result[i][j])
    key_result.append(par)
key_result

[[],
 [],
 ['虎迷'],
 ['留言',
  '機會',
  '機票',
  '活動',
  '活動',
  '時間',
  'Tigerair',
  'Taiwan',
  '留言',
  '留言',
  '留言',
  '台灣虎航',
  '機會',
  '參加',
  '抽獎',
  '台灣虎航',
  '航班',
  '體驗',
  '注意事項',
  '留言',
  '參加',
  '抽獎',
  '機會',
  '獲得',
  '台灣虎航',
  '航班',
  '來回',
  '機票',
  '2015',
  '使用',
  '中獎',
  '手機',
  '台灣虎航',
  '機票',
  '得獎',
  '機會',
  '活動',
  '相關',
  '機票',
  '中獎',
  '活動',
  '參加',
  '活動',
  '參加',
  '活動'],
 ['機場'],
 ['虎迷', '參加', '活動', '台灣虎航', '虎迷', '活動', 'http'],
 ['免費', '來回', '機票', '參加', '活動', 'http'],
 ['2015', '2015', '虎迷'],
 ['澳門', '旅遊', '小時', '當地', '當地', '參加', '澳門', '旅遊', 'http'],
 ['服務', '優惠', '航班', '服務', '機票'],
 ['中獎', '訊息', '手機', '台灣虎航', '機票'],
 ['9月', '台北', '航線', '台灣虎航', '航班', '體驗', '機票', '台灣虎航', '9月', '台北', '航班'],
 ['台灣虎航',
  '訂位',
  '優惠',
  '台灣虎航',
  '機票',
  '說明',
  '台灣虎航',
  'tigerair',
  'tw',
  '台灣虎航',
  '台灣虎航',
  '銷售',
  '售價',
  '機場稅',
  '訂位',
  '服務費',
  '說明',
  '虎迷',
  '購票',
  '航線',
  '促銷',
  '訊息',
  '台灣虎航',
  '台灣虎航',
  '資訊',
  '優惠',
  '台灣虎航',
  '台灣虎航',
  '優惠'],
 ['台灣虎航', '台灣虎航

In [183]:
T_test_model = word2vec.Word2Vec(key_result, size=300, min_count=10)
T_model.save("model/Tigerair_key_model.bin")

In [184]:
key_model = word2vec.Word2Vec.load("model/Tigerair_key_model.bin")
word_vectors = key_model.wv

In [185]:
res = word_vectors.wv.most_similar('虎迷', topn = 20)

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [186]:
res

[('分享', 0.902275562286377),
 ('可以', 0.898814857006073),
 ('準備', 0.8933571577072144),
 ('就是', 0.8924463987350464),
 ('體驗', 0.8862548470497131),
 ('住宿', 0.8798786401748657),
 ('機會', 0.8730891942977905),
 ('朋友', 0.8655794262886047),
 ('旅行', 0.8628038763999939),
 ('免費', 0.8601059913635254),
 ('自己', 0.856334924697876),
 ('機票', 0.8543732762336731),
 ('出國', 0.8455445170402527),
 ('前往', 0.8442637324333191),
 ('不要', 0.8434194326400757),
 ('折價券', 0.8433970212936401),
 ('參加', 0.8433293104171753),
 ('一張', 0.8369359970092773),
 ('現場', 0.8341183662414551),
 ('不用', 0.8272459506988525)]